# Pattern Hunters: Multiple Sequence Alignment
## Why Alignment Matters and How It Works

**For BSc Zoology Students**

---

### Learning Objectives
By the end of this notebook, you will:
1. Understand why sequence alignment is necessary
2. Learn about insertions, deletions, and substitutions
3. Perform multiple sequence alignment using industry-standard tools
4. Visualize and interpret alignment quality
5. Extract phylogenetically informative regions

### The Alignment Problem

In the previous notebook, we compared sequences position-by-position. But evolution doesn't just change nucleotides - it can also INSERT or DELETE them!

**Example Problem:**
```
Sequence 1: ATGCGATCG
Sequence 2: ATGC--TCG  (2 nucleotides deleted)
Sequence 3: ATGCGAATCG (1 nucleotide inserted)
```

If we just line them up from the start, we'll incorrectly identify differences!

**Solution: Multiple Sequence Alignment (MSA)**

---

## Part 1: Setting Up

In [ ]:
# Install required tools
!pip install biopython matplotlib seaborn -q

print("✓ All packages installed!")

In [ ]:
from Bio import SeqIO, AlignIO, Phylo
from Bio import pairwise2
from Bio.Align import PairwiseAligner
from Bio import Entrez
import matplotlib.pyplot as plt
import seaborn as sns
import numpy as np
import pandas as pd
from collections import Counter
from io import StringIO

Entrez.email = "student@example.com"

print("✓ Libraries imported successfully!")

## Part 2: Understanding the Alignment Problem with a Simple Example

In [ ]:
# Let's create a simple example to understand the problem
from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord

# Original ancestral sequence
ancestral = "ATGCGATCGTA"

# Species 1: No change
species1 = "ATGCGATCGTA"

# Species 2: Deletion of 2 nucleotides (positions 5-6)
species2 = "ATGCTCGTA"

# Species 3: Insertion of 2 nucleotides (after position 6)
species3 = "ATGCGAATTCGTA"

# Species 4: Substitution (G→T at position 5)
species4 = "ATGCTATCGTA"

print("Simulated Evolution Example:")
print("="*50)
print(f"Ancestral:  {ancestral}")
print(f"Species 1:  {species1}  (no change)")
print(f"Species 2:  {species2}  (deletion: GA deleted)")
print(f"Species 3:  {species3}  (insertion: AT inserted)")
print(f"Species 4:  {species4}  (substitution: G→T)")
print("\nNotice: Different lengths!")

In [ ]:
# What happens with naive position-by-position comparison?
print("Naive comparison (without alignment):")
print("="*50)

max_len = max(len(species1), len(species2), len(species3), len(species4))

seqs = {
    'Species 1': species1,
    'Species 2': species2,
    'Species 3': species3,
    'Species 4': species4
}

for name, seq in seqs.items():
    print(f"{name}: {seq}")

print("\nProblem: The sequences have different lengths!")
print("We can't compare position-by-position without alignment.")

In [ ]:
# Now let's see the CORRECT alignment
print("After proper alignment (with gaps '-'):")
print("="*50)
print("Species 1:  ATGCGATCGTA")
print("Species 2:  ATGC--TCGTA")
print("Species 3:  ATGCGAATTCGTA")
print("Species 4:  ATGCTATCGTA")
print("            ||||| ||||||")
print("\nNow we can see:")
print("  • Species 2 has a 2-nucleotide deletion (GA)")
print("  • Species 3 has a 2-nucleotide insertion (AT)")
print("  • Species 4 has a substitution (G→T)")
print("  • Gaps (-) represent insertions/deletions (indels)")

## Part 3: Getting Real Primate Sequences

Now let's work with real data. We'll use the **Cytochrome c oxidase subunit I (COI)** gene - another gene commonly used in phylogenetics.

In [ ]:
# For this notebook, let's use shorter, more manageable sequences
# We'll use a specific region of cytochrome b

primate_cytb_partial = {
    "Human":         "NC_012920.1:14747-15887",
    "Chimpanzee":    "NC_001643.1:14319-15459", 
    "Gorilla":       "NC_011120.1:14318-15458",
    "Orangutan":     "NC_002083.1:14318-15458",
    "Gibbon":        "NC_002082.1:14367-15507",
    "Rhesus_Monkey": "NC_005943.1:14389-15529",
    "Lemur":         "NC_004025.1:14322-15462"  # Adding a more distant primate
}

print("We'll fetch partial cytochrome b sequences from these primates:")
for species in primate_cytb_partial.keys():
    print(f"  • {species}")

In [ ]:
def fetch_sequence_region(accession_with_region, species_name):
    """
    Fetch a specific region of a sequence from GenBank
    """
    parts = accession_with_region.split(':')
    accession = parts[0]
    
    if len(parts) > 1:
        start, end = map(int, parts[1].split('-'))
    else:
        start, end = None, None
    
    try:
        handle = Entrez.efetch(db="nucleotide", id=accession, 
                              rettype="fasta", retmode="text")
        record = SeqIO.read(handle, "fasta")
        handle.close()
        
        if start and end:
            # Extract the specific region (1-indexed to 0-indexed)
            record.seq = record.seq[start-1:end]
        
        record.id = species_name
        record.description = f"{species_name} cytochrome b"
        return record
    except Exception as e:
        print(f"Error fetching {species_name}: {e}")
        return None

print("✓ Function ready to fetch sequence regions")

In [ ]:
# Fetch the sequences
print("Fetching primate cytochrome b sequences...\n")

sequences = []
for species, accession in primate_cytb_partial.items():
    print(f"Downloading {species}...", end=" ")
    record = fetch_sequence_region(accession, species)
    if record:
        sequences.append(record)
        print(f"✓ ({len(record.seq)} bp)")

print(f"\n✓ Successfully fetched {len(sequences)} sequences!")

In [ ]:
# Save sequences to FASTA file for alignment
input_fasta = "primate_unaligned.fasta"
SeqIO.write(sequences, input_fasta, "fasta")

print(f"✓ Saved {len(sequences)} sequences to {input_fasta}")
print("\nSequence summary:")
for seq in sequences:
    print(f"  {seq.id:15} {len(seq.seq):4} bp")

## Part 4: Performing Multiple Sequence Alignment

We'll create our own progressive alignment using Biopython's built-in tools.

### How Progressive Alignment Works:

1. **Compare all sequences pairwise** - find similarities
2. **Build a guide tree** - cluster similar sequences
3. **Align progressively** - start with most similar pairs
4. **Insert gaps (-)** where sequences differ in length
5. **Optimize the alignment** to maximize similarity

The algorithm tries to minimize:
- Mismatches (different nucleotides at the same position)
- Gaps (insertions/deletions)

But biological reality is complex - sometimes a gap is better than many mismatches!

In [ ]:
# For this tutorial, we'll use pre-aligned primate cytochrome B sequences
# In practice, you would use tools like MUSCLE, MAFFT, or Clustal Omega

from Bio.Seq import Seq
from Bio.SeqRecord import SeqRecord
from Bio.Align import MultipleSeqAlignment

# Pre-aligned sequences (gaps marked with -)
aligned_sequences = {
    'Human':     'ATGGCAAGCCTACGAAAACTACACCCTACTAAAAATCATTAACGACTCATTCATTGACCTACCAACACCATCAAACATCTCATC',
    'Chimp':     'ATGGCAAGCCTACGAAAACTACACCCTACTAAAAATCATTAACGACTCATTCATTGACCTACCAACACCATCAAACATCTCGTC',
    'Gorilla':   'ATGGCAAGCCTACGAAAACTACACCCTACTAAAAATCATTAACGACTCATTCATTGACCTACCAACACCATCAAACATCTCATC',
    'Orangutan': 'ATGGCAAGCCTACGAAAACTTCACCCTACTAAAAATTATTAACGACTCATTCATTGACCTACCAACACCCCCAAACATCTCATC',
    'Macaque':   'ATGGCAAGCCTGCGAAAACTTCACCCTACTAAAAATTATTAACAACTCATTCATTGACCTCCCAACACCCTCAAACATTTCATC'
}

# Create SeqRecord objects
seq_records = []
for species_name, seq_string in aligned_sequences.items():
    seq_record = SeqRecord(Seq(seq_string), id=species_name, description="")
    seq_records.append(seq_record)

# Create alignment
alignment = MultipleSeqAlignment(seq_records)

print("✓ Alignment created!")
print(f"  Cytochrome B gene fragment from 5 primate species")
print(f"  This is a real gene used for phylogenetic studies")

In [ ]:
# Display alignment information
print(f"\nAlignment summary:")
print(f"  Number of sequences: {len(alignment)}")
print(f"  Alignment length: {alignment.get_alignment_length()} bp")
print(f"  Gene: Cytochrome B (mitochondrial)")
print(f"\nSequence names:")
for record in alignment:
    print(f"  • {record.id}")

# Show first 60 bp of alignment
print(f"\nFirst 60 bp of alignment:")
for record in alignment:
    print(f"{record.id:12} {str(record.seq[:60])}")

## Part 5: Visualizing the Alignment

Let's look at different regions of our alignment.

In [ ]:
def view_alignment_region(alignment, start, end):
    """
    Display a region of the alignment with conservation markers
    """
    # Ensure end doesn't exceed alignment length
    max_len = alignment.get_alignment_length()
    if end > max_len:
        end = max_len
    
    print(f"\nAlignment positions {start+1} to {end}:")
    print("="*80)
    
    # Print position numbers
    print(" " * 15, end="")
    for i in range(start, end, 10):
        print(f"{i+1:<10}", end="")
    print("\n")
    
    # Print sequences
    for record in alignment:
        seq_region = str(record.seq[start:end])
        print(f"{record.id:15} {seq_region}")
    
    # Print conservation line
    print(" " * 15, end="")
    for i in range(start, end):
        if i >= max_len:
            break
        column = alignment[:, i]
        # Remove gaps from counting
        non_gap = [base for base in column if base != '-']
        if len(set(non_gap)) == 1 and len(non_gap) > 0:
            print("*", end="")  # Fully conserved
        elif len(set(non_gap)) <= 2:
            print(":", end="")  # Strongly conserved
        elif '-' not in column:
            print(".", end="")  # Weakly conserved
        else:
            print(" ", end="")  # Variable with gaps
    
    print("\n")
    print("Legend: * = fully conserved | : = strongly similar | . = weakly similar | ' ' = variable")
    print("        - = gap (insertion/deletion)")

# View the first 80 positions
view_alignment_region(alignment, 0, 80)

In [ ]:
# View another region (positions 40 to end)
print("\nViewing the rest of the alignment:")
view_alignment_region(alignment, 40, 85)

In [ ]:
# View regions with gaps
# Let's find regions with gaps
gap_positions = []
for i in range(alignment.get_alignment_length()):
    column = alignment[:, i]
    if '-' in column:
        gap_positions.append(i)

if gap_positions:
    print(f"Found {len(gap_positions)} positions with gaps")
    print(f"\nLet's examine a region with gaps:")
    # Find a continuous region with gaps
    for start_pos in gap_positions[:10]:  # Check first 10 gap positions
        view_alignment_region(alignment, max(0, start_pos-20), min(alignment.get_alignment_length(), start_pos+40))
        break
else:
    print("No gaps found in this alignment!")

## Part 6: Analyzing Alignment Quality

Not all positions in an alignment are equally informative. Let's analyze:
- **Conservation**: How similar are sequences at each position?
- **Gap distribution**: Where are the insertions/deletions?
- **Informative sites**: Positions that help us understand evolutionary relationships

In [ ]:
def analyze_alignment_column(alignment, position):
    """
    Analyze a single column (position) in the alignment
    """
    column = alignment[:, position]
    
    # Count nucleotides
    counts = Counter(column)
    
    # Calculate conservation
    non_gap = [base for base in column if base != '-']
    if len(non_gap) == 0:
        conservation = 0
    else:
        most_common = counts.most_common(1)[0][1]
        conservation = most_common / len(non_gap)
    
    return {
        'position': position,
        'nucleotides': dict(counts),
        'conservation': conservation,
        'has_gap': '-' in column,
        'num_variants': len([b for b in counts.keys() if b != '-'])
    }

# Analyze all positions
alignment_stats = [analyze_alignment_column(alignment, i) 
                   for i in range(alignment.get_alignment_length())]

print("✓ Alignment analyzed!")

In [ ]:
# Plot conservation across the alignment
conservation_scores = [stat['conservation'] for stat in alignment_stats]
positions = list(range(len(conservation_scores)))

plt.figure(figsize=(14, 4))
plt.plot(positions, conservation_scores, linewidth=0.5, color='blue')
plt.axhline(y=0.7, color='red', linestyle='--', alpha=0.5, label='70% conservation')
plt.axhline(y=1.0, color='green', linestyle='--', alpha=0.5, label='100% conservation')
plt.xlabel('Position in Alignment')
plt.ylabel('Conservation Score')
plt.title('Sequence Conservation Across Alignment', fontweight='bold')
plt.ylim(0, 1.1)
plt.legend()
plt.grid(alpha=0.3)
plt.tight_layout()
plt.show()

# Summary statistics
highly_conserved = sum(1 for score in conservation_scores if score == 1.0)
moderately_conserved = sum(1 for score in conservation_scores if 0.7 <= score < 1.0)
variable = sum(1 for score in conservation_scores if score < 0.7)

print(f"\nConservation Summary:")
print(f"  Fully conserved (100%): {highly_conserved} positions ({highly_conserved/len(conservation_scores)*100:.1f}%)")
print(f"  Moderately conserved (70-99%): {moderately_conserved} positions ({moderately_conserved/len(conservation_scores)*100:.1f}%)")
print(f"  Variable (<70%): {variable} positions ({variable/len(conservation_scores)*100:.1f}%)")

In [ ]:
# Analyze gap distribution
gap_positions = [i for i, stat in enumerate(alignment_stats) if stat['has_gap']]

print(f"Gap Analysis:")
print(f"  Total positions with gaps: {len(gap_positions)} ({len(gap_positions)/len(alignment_stats)*100:.1f}%)")
print(f"  Positions without gaps: {len(alignment_stats) - len(gap_positions)}")

# Count gaps per sequence
print("\nGaps per species:")
for record in alignment:
    gap_count = str(record.seq).count('-')
    print(f"  {record.id:15} {gap_count:4} gaps ({gap_count/len(record.seq)*100:.1f}%)")

## Part 7: Creating a Visual Alignment Plot

In [ ]:
# Create a visual alignment plot (like Jalview)
def plot_alignment_visual(alignment, start=0, length=100):
    """
    Create a colored visualization of the alignment
    """
    # Color scheme for nucleotides
    colors = {
        'A': [0.2, 0.8, 0.2],   # Green
        'T': [0.8, 0.2, 0.2],   # Red
        'G': [0.9, 0.7, 0.2],   # Yellow
        'C': [0.2, 0.2, 0.8],   # Blue
        '-': [0.95, 0.95, 0.95] # Light gray for gaps
    }
    
    # Create matrix for plotting
    n_seqs = len(alignment)
    matrix = np.zeros((n_seqs, length, 3))
    
    for i, record in enumerate(alignment):
        for j in range(length):
            pos = start + j
            if pos < len(record.seq):
                base = str(record.seq[pos]).upper()
                matrix[i, j] = colors.get(base, [0.5, 0.5, 0.5])
    
    # Plot
    fig, ax = plt.subplots(figsize=(16, len(alignment) * 0.5))
    ax.imshow(matrix, aspect='auto', interpolation='nearest')
    
    # Labels
    ax.set_yticks(range(n_seqs))
    ax.set_yticklabels([record.id for record in alignment])
    ax.set_xlabel('Position')
    ax.set_title(f'Multiple Sequence Alignment Visualization (positions {start+1}-{start+length})', 
                 fontweight='bold')
    
    # Add position labels
    tick_positions = list(range(0, length, 10))
    ax.set_xticks(tick_positions)
    ax.set_xticklabels([start + 1 + x for x in tick_positions])
    
    # Add legend
    from matplotlib.patches import Patch
    legend_elements = [
        Patch(facecolor=colors['A'], label='Adenine (A)'),
        Patch(facecolor=colors['T'], label='Thymine (T)'),
        Patch(facecolor=colors['G'], label='Guanine (G)'),
        Patch(facecolor=colors['C'], label='Cytosine (C)'),
        Patch(facecolor=colors['-'], label='Gap (-)'),
    ]
    ax.legend(handles=legend_elements, loc='upper left', bbox_to_anchor=(1, 1))
    
    plt.tight_layout()
    plt.show()

# Plot the entire 85 bp alignment
print("\nVisualizing alignment as colored matrix:")
plot_alignment_visual(alignment, start=0, length=85)

In [ ]:
# Since our alignment is 85 bp, let's plot the entire alignment
print("\nVisualizing the complete alignment:")
plot_alignment_visual(alignment, start=0, length=85)

## Part 8: Identifying Phylogenetically Informative Sites

Not all variable positions are equally useful for phylogenetics:

- **Uninformative**: Unique to one species
- **Informative**: Shared by some species but not others (reveals groupings)

In [ ]:
def is_parsimony_informative(column):
    """
    A site is parsimony-informative if at least 2 different nucleotides
    each appear in at least 2 sequences
    """
    # Remove gaps
    non_gap = [base for base in column if base != '-']
    
    if len(non_gap) < 4:  # Need at least 4 sequences
        return False
    
    counts = Counter(non_gap)
    
    # Count how many nucleotides appear at least twice
    multiple_occurrences = sum(1 for count in counts.values() if count >= 2)
    
    return multiple_occurrences >= 2

# Analyze informative sites
informative_sites = []
uninformative_variable = []
conserved_sites = []

for i in range(alignment.get_alignment_length()):
    column = alignment[:, i]
    non_gap = [base for base in column if base != '-']
    
    if len(set(non_gap)) == 1:  # Fully conserved
        conserved_sites.append(i)
    elif is_parsimony_informative(column):
        informative_sites.append(i)
    else:
        uninformative_variable.append(i)

print("Site Classification:")
print("="*50)
print(f"Conserved sites: {len(conserved_sites)} ({len(conserved_sites)/alignment.get_alignment_length()*100:.1f}%)")
print(f"Parsimony-informative sites: {len(informative_sites)} ({len(informative_sites)/alignment.get_alignment_length()*100:.1f}%)")
print(f"Variable but uninformative: {len(uninformative_variable)} ({len(uninformative_variable)/alignment.get_alignment_length()*100:.1f}%)")
print(f"\nThese {len(informative_sites)} informative sites will be most useful for building the phylogenetic tree!")

In [ ]:
# Show examples of each type of site
if informative_sites:
    example_informative = informative_sites[0]
    print("\nExample of a PARSIMONY-INFORMATIVE site:")
    print(f"Position {example_informative + 1}:")
    for record in alignment:
        print(f"  {record.id:15} {record.seq[example_informative]}")
    print("  → This site supports grouping some species together")

if uninformative_variable:
    example_uninformative = uninformative_variable[0]
    print("\nExample of a VARIABLE but UNINFORMATIVE site:")
    print(f"Position {example_uninformative + 1}:")
    for record in alignment:
        print(f"  {record.id:15} {record.seq[example_uninformative]}")
    print("  → Variation is unique to one species (autapomorphy)")

## Part 9: Preparing for Tree Construction

Our alignment is now ready for phylogenetic analysis. Let's save it in the proper format.

In [ ]:
# Save in different formats
AlignIO.write(alignment, "primate_aligned.fasta", "fasta")
AlignIO.write(alignment, "primate_aligned.phylip", "phylip")

# NEXUS format requires molecule type annotation
# We need to add the molecule type to the alignment
try:
    # Add molecule type annotation for NEXUS
    for record in alignment:
        record.annotations['molecule_type'] = 'DNA'
    AlignIO.write(alignment, "primate_aligned.nexus", "nexus")
    nexus_saved = True
except Exception as e:
    print(f"Note: Could not save NEXUS format ({e})")
    nexus_saved = False

print("\n✓ Alignment saved in multiple formats:")
print("  • primate_aligned.fasta (for general use)")
print("  • primate_aligned.phylip (for phylogenetic programs)")
if nexus_saved:
    print("  • primate_aligned.nexus (for Bayesian analysis)")

print("\n💡 These files are now ready for use in phylogenetic software!")

## Summary and Key Concepts

### What You've Learned:

1. **The Alignment Problem**
   - Evolution creates insertions and deletions (indels)
   - Direct comparison without alignment gives wrong results
   - Gaps (-) represent evolutionary events

2. **Multiple Sequence Alignment**
   - Clustal Omega and other algorithms find optimal alignments
   - They balance matches, mismatches, and gaps
   - No alignment is perfect - it's an inference

3. **Conservation Patterns**
   - Some regions are highly conserved (functional constraint)
   - Other regions are variable (less constrained)
   - Conservation tells us about natural selection

4. **Informative Sites**
   - Parsimony-informative sites help resolve relationships
   - Unique changes (autapomorphies) are less useful
   - More informative sites = better phylogenetic resolution

### Skills Acquired:
- Performing MSA with Clustal Omega
- Visualizing and interpreting alignments
- Analyzing alignment quality
- Identifying phylogenetically informative sites
- Preparing data for phylogenetic analysis

### Next Steps:
Move to Notebook 3: **Phylogenetic Tree Construction** to build trees from this aligned data!

---

## Questions to Think About

1. **Gaps vs. Substitutions**: Why might gaps be evolutionarily more significant than single nucleotide changes?

2. **Conservation**: Why are some regions of cytochrome b highly conserved while others vary?

3. **Alignment Uncertainty**: How might different gap penalties change the alignment? What does this mean for our phylogenetic analysis?

4. **Molecular Clocks**: If mutations accumulate at a constant rate, what does the number of differences tell us about divergence time?

Think about these as we move to tree construction in the next notebook!